In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = 15.7,13.27
plt.rcParams['font.family'] = 'AppleGothic'

In [35]:
from src.model import BasicModel

class LEARN_INFO:
    def __init__(self, name, columns, is_contain_cluster_label=False):
        self.name = name
        self.columns = columns
        self.is_label = is_contain_cluster_label
    

# INPUT_REPORT_SIMULATION = [
#     LEARN_INFO(name="univariate", columns=['energy (kw 15min)'], is_contain_cluster_label=False),
#     LEARN_INFO(name="with cluster label", columns=['energy (kw 15min)'], is_contain_cluster_label=True),
# ]

DEFAULT_COLUMNS = [
    'energy (kw 15min)',
    'week sin',
    'week cos',
    'year sin',
    'year cos',
    'season idx',
    'avg ta',
    'avg rhm'
]

_ = {
    "univariate": ['energy (kw 15min)'],
    "multivariate (week)": [
    'energy (kw 15min)',
    'week sin',
    'week cos'],
    "multivariate (year)": [
        'energy (kw 15min)',
        'year sin',
        'year cos',
    ],
    "multivariate (avg ta)": [
        'energy (kw 15min)',
        'avg ta'
    ],
    "multivariate (avg rhm)": [
        'energy (kw 15min)',
        'avg rhm'
    ],
    "multivariate (season)": [
        'energy (kw 15min)',
        'season idx'
    ]
}

# Input Report Simulation
IRS = []
for key in _.keys():
    IRS.append(
        LEARN_INFO(name=key, 
                   columns=_[key], 
                   is_contain_cluster_label=False),
    )

# Training Model Array 
TMA = []


In [36]:
is_switch = False

for _ in IRS:
    bm = BasicModel(name=_.name,
                   columns=_.columns,
                   is_contain_cluster_label=_.is_label,
                   is_switch=is_switch)
    
    bm.set_window()
    bm.set_model()
    print(bm)
    bm.training()
    
    TMA.append(bm)

###### Basic Model Information ######
 - name : multivariate (season)
 - columns : energy (kw 15min),season idx
 - datas : 
                     energy (kw 15min)  season idx
Date Time                                         
2017-01-01 00:00:00              15250           4
2017-01-01 01:00:00              14000           4
2017-01-01 02:00:00              14750           4
2017-01-01 03:00:00              29500           4
2017-01-01 04:00:00              15750           4
 - norm_datas : 
                     energy (kw 15min)  season idx
Date Time                                         
2017-01-01 00:00:00          -0.719354    1.351672
2017-01-01 01:00:00          -0.811090    1.351672
2017-01-01 02:00:00          -0.756048    1.351672
2017-01-01 03:00:00           0.326432    1.351672
2017-01-01 04:00:00          -0.682660    1.351672

Epoch 1/50
274/274 [==============================] - 2s 4ms/step - loss: 0.4807 - mean_absolute_error: 0.5381 - val_loss: 0.2990 - val_mean_abs

In [44]:
import pandas as pd
evaluate_df = pd.DataFrame();

for _ in TMA:
    _.set_predict(is_reshape=True)
    evaluate_df = evaluate_df.append(_.statistic(), ignore_index=True)

###### [Notice] set predict info start ###### 

0 / 171
50 / 171
100 / 171
150 / 171
171 / 171 complete.

###### [Notice] set predict info success ###### 

###### [Notice] set predict info start ###### 

0 / 171
50 / 171
100 / 171
150 / 171
171 / 171 complete.

###### [Notice] set predict info success ###### 

###### [Notice] set predict info start ###### 

0 / 171
50 / 171
100 / 171
150 / 171
171 / 171 complete.

###### [Notice] set predict info success ###### 

###### [Notice] set predict info start ###### 

0 / 171
50 / 171
100 / 171
150 / 171
171 / 171 complete.

###### [Notice] set predict info success ###### 

###### [Notice] set predict info start ###### 

0 / 171
50 / 171
100 / 171
150 / 171
171 / 171 complete.

###### [Notice] set predict info success ###### 

###### [Notice] set predict info start ###### 

0 / 171
50 / 171
100 / 171
150 / 171
171 / 171 complete.

###### [Notice] set predict info success ###### 



In [46]:
evaluate_df.index = [_.name for _ in TMA]
evaluate_df

,ecv,mae,mean dis,mean sim,mse,tss,wss
univariate,84.114023,0.173580,1.078234,0.943865,0.064451,1456.910039,231.444388
multivariate (week),84.566585,0.177364,1.063015,0.944613,0.062615,1456.910039,224.850977
multivariate (year),71.023732,0.277712,1.516890,0.906415,0.117560,1456.910039,422.158161
multivariate (avg ta),69.115246,0.274243,1.534108,0.915134,0.125303,1456.910039,449.963078
multivariate (avg rhm),83.608547,0.179479,1.101039,0.942839,0.066502,1456.910039,238.808725
multivariate (season),78.560563,0.205254,1.264982,0.931415,0.086982,1456.910039,312.353310
